In [7]:
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import cv2
import datetime
import dateutil.relativedelta
import math

In [8]:
def draw_xywha(im, num, x, y, w, h, angle, color=(0,255,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    font = cv2.FONT_HERSHEY_SIMPLEX
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w, -h], [w, -h], [w, h], [-w, h]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([2*x, 2*y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    cv2.putText(im,str(num),(int(2 *x)-15, int(2 *y)), font, 2, (0, 255, 0), 3, cv2.LINE_AA)

def drawing_bb(folder_path, video_name, RP1, RP2):

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v') ,5, (1024, 1024))
    allFiles = os.listdir(folder_path)
    jsonList = []
    files = []
    RP = RP1 + RP2
    RP = sorted(RP)
    RP_index = 0
    for f in RP:
        if '._' in f:
            RP.remove(f)
    for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
        if '.json' in f:
            if '._' not in f:
                jsonList.append(f)
#     print(files)
    jsonList = sorted(jsonList)
    files = sorted(files)
#     print(len(jsonList), len(files))
    RP1count = 0
    RP2count = 0
    for i in range(len(files)):
        loc =  folder_path + files[i]
        jloc = folder_path + jsonList[i]
        with open(jloc) as f:
            cam1 = json.load(f)
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.imread(loc)
        boxes = list(cam1["Bounding Box"].keys())
        count = 0
        for b in boxes:
            count+=1
            draw_xywha(image, count, cam1["Bounding Box"][b][0], cam1["Bounding Box"][b][1], cam1["Bounding Box"][b][2], cam1["Bounding Box"][b][3], cam1["Bounding Box"][b][4], color=(0,255,0), linewidth=5)
#             cv2.putText(image,str(count),(int(cam1["Bounding Box"][b][0])-15, int(cam1["Bounding Box"][b][1])), font, 2, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image,"People#: " + str(cam1["Num People"]),(1600, 50), font, 2, (0, 0, 255), 3, cv2.LINE_AA)
        ts = datetime.datetime.fromtimestamp(1618919891.8548079)
        ts2 = datetime.datetime.fromtimestamp(float(cam1['Timestamp']))
        diff = dateutil.relativedelta.relativedelta(ts2, ts)
        print(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60), (diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60)    
        cv2.putText(image,"since: " + str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60))+"min "+str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60))+"sec",(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        
        RP_relative = float(RP[RP_index].replace('.json', ''))
        if float(cam1['Timestamp']) > RP_relative:
            if RP[RP_index] in RP1:
                with open('/home/team19/Fusion/RP1/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP1count = int(RPinfo['count'])
            elif RP[RP_index] in RP2:
                with open('/home/team19/Fusion/RP2/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP2count = int(RPinfo['count'])
        cv2.putText(image, "TDS1: " + str(RP1count),(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.putText(image, "TDS2: " + str(RP2count),(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        resized = cv2.resize(image, (1024,1024))
        out.write(resized)
        
#     for data_file in sorted(files):
#         loc = folder_path+data_file
#         print(loc)
#         image = cv2.imread(loc)
        
#         resized = cv2.resize(image, (1024,1024))

#         out.write(resized)

        
    out.release()

In [ ]:
folder_path = '/home/team19/Desktop/Axis_DL/Detection/YOLO/04-20-2021-07:58:10/Camera 1/'
video_name = 'trial.mp4'
RP1 = os.listdir('/home/team19/Fusion/RP1/')
RP2 = os.listdir('/home/team19/Fusion/RP2/')
drawing_bb(folder_path, video_name, RP1, RP2)

0 0
0 1
0 3
0 5
0 7
0 9
0 11
0 13
0 15
0 16
0 18
0 20
0 22
0 24
0 26
0 28
0 30
0 32
0 33
0 35
0 37
0 39
0 41
0 43
0 45
0 46
0 48
0 50
0 52
0 54
0 56
0 58
1 0
1 1
1 3
1 5
1 7
1 9
1 11
1 13
1 15
1 17
1 18
1 20
1 22
1 24
1 26
1 28
1 30
1 32
1 34
1 36
1 37
1 39
1 41
1 43
1 45
1 47
1 49
1 51
1 52
1 54
1 56
1 58
2 0
2 2
2 4
2 6
2 8
2 9
2 11
2 13
2 15
2 17
2 19
2 21
2 23
2 25
2 27
2 28
2 30
2 32
2 34
2 36
2 38
2 40
2 42
2 43
2 45
2 47
2 49
2 51
2 53
2 55
2 56
2 58
3 0
3 2
3 4
3 6
3 8
3 10
3 11
3 13
3 15
3 17
3 19
3 21
3 23
3 25
3 27
3 28
3 30
3 32
3 34
3 36
3 38
3 40
3 42
3 44
3 45
3 47
3 49
3 51
3 53
3 55
3 57
3 58
4 0
4 2
4 4
4 6
4 8
4 10
4 12
4 14
4 15
4 17
4 19
4 21
4 23
4 25
4 27
4 29
4 31
4 33
4 35
4 37
4 39
4 40
4 42
4 44
4 46
4 48
4 50
4 52
4 53
4 55
4 57
4 59
5 1
5 3
5 5
5 7
5 9
5 11
5 13
5 14
5 16
5 18
5 20
5 22
5 24
5 26
5 28
5 30
5 32
5 34
5 36
5 37
5 39
5 41
5 43
5 45
5 47
5 49
5 51
5 53
5 54
5 56
5 58
6 0
6 2
6 4
6 6
6 8
6 10
6 12
6 13
6 15
6 17
6 19
6 21
6 23
6 25
6 27
6 29
6 3

46 4
46 6
46 8
46 10
46 12
46 14
46 16
46 17
46 19
46 21
46 23
46 25
46 27
46 29
46 31
46 33
46 35
46 37
46 38
46 40
46 42
46 44
46 46
46 48
46 50
46 52
46 53
46 55
46 57
46 59
47 1
47 3
47 5
47 7
47 8
47 10
47 12
47 14
47 16
47 18
47 20
47 22
47 24
47 26
47 28
47 29
47 31
47 33
47 35
47 37
47 39
47 41
47 43
47 44
47 46
47 48
47 50
47 52
47 54
47 56
47 58
48 0
48 2
48 3
48 5
48 7
48 9
48 11
48 13
48 15
48 17
48 19
48 20
48 22
48 24
48 26
48 28
48 30
48 32
48 34
48 35
48 37
48 39
48 41
48 43
48 45
48 47
48 49
48 51
48 53
48 54
48 56
48 58
49 0
49 2
49 4
49 6
49 8
49 10
49 11
49 13
49 15
49 17
49 19
49 21
49 23
49 24
49 26
49 28
49 30
49 32
49 34
49 36
49 38
49 40
49 42
49 43
49 45
49 47
49 49
49 51
49 53
49 55
49 57
49 59
50 1
50 3
50 5
50 6
50 8
50 10
50 12
50 14
50 16
50 18
50 20
50 22
50 24
50 26
50 27
50 29
50 31
50 33
50 35
50 37
50 39
50 41
50 43
50 45
50 46
50 48
50 50
50 52
50 54
50 56
50 58
51 0
51 2
51 4
51 6
51 7
51 9
51 11
51 13
51 15
51 17
51 19
51 21
51 23
51 25
51 26
51 2

90 42
90 44
90 46
90 48
90 50
90 52
90 54
90 55
90 57
90 59
91 1
91 3
91 5
91 7
91 9
91 11
91 12
91 14
91 16
91 18
91 20
91 22
91 24
91 26
91 28
91 30
91 32
91 33
91 35
91 37
91 39
91 41
91 43
91 45
91 47
91 49
91 51
91 52
91 54
91 56
91 58
92 0
92 2
92 4
92 6
92 8
92 10
92 12
92 13
92 15
92 17
92 19
92 21
92 23
92 25
92 27
92 29
92 31
92 33
92 34
92 36
92 38
92 40
92 42
92 44
92 46
92 48
92 50
92 52
92 54
92 56
92 57
92 59
93 1
93 3
93 5
93 7
93 9
93 11
93 13
93 15
93 17
93 18
93 20
93 22
93 24
93 26
93 28
93 30
93 32
93 34
93 36
93 37
93 39
93 41
93 43
93 45
93 47
93 49
93 51
93 53
93 54
93 56
93 58
94 0
94 2
94 4
94 6
94 8
94 10
94 12
94 14
94 15
94 17
94 19
94 21
94 23
94 25
94 27
94 29
94 31
94 33
94 35
94 37
94 38
94 40
94 42
94 44
94 46
94 48
94 50
94 52
94 54
94 56
94 58
95 0
95 2
95 4
95 6
95 7
95 9
95 11
95 13
95 15
95 17
95 19
95 21
95 23
95 25
95 27
95 29
95 31
95 33
95 34
95 36
95 38
95 40
95 42
95 44
95 46
95 48
95 50
95 52
95 53
95 55
95 57
95 59
96 1
96 3
96 5
96 7
96 9

130 26
130 28
130 30
130 32
130 34
130 36
130 38
130 39
130 41
130 43
130 45
130 47
130 49
130 51
130 53
130 55
130 57
130 59
131 0
131 2
131 4
131 6
131 8
131 10
131 12
131 14
131 16
131 18
131 20
131 22
131 23
131 25
131 27
131 29
131 31
131 33
131 35
131 37
131 39
131 41
131 43
131 44
131 46
131 48
131 50
131 52
131 54
131 56
131 58
132 0
132 2
132 4
132 5
132 7
132 9
132 11
132 13
132 15
132 17
132 19
132 21
132 23
132 25
132 26
132 28
132 30
132 32
132 34
132 36
132 38
132 40
132 42
132 44
132 46
132 48
132 50
132 51
132 53
132 55
132 57
132 59
133 1
133 3
133 5
133 7
133 9
133 11
133 12
133 14
133 16
133 18
133 20
133 22
133 24
133 26
133 28
133 30
133 32
133 34
133 35
133 37
133 39
133 41
133 43
133 45
133 47
133 49
133 51
133 53
133 55
133 56
133 58
134 0
134 2
134 4
134 6
134 8
134 10
134 12
134 14
134 16
134 18
134 20
134 21
134 23
134 25
134 27
134 29
134 31
134 33
134 35
134 37
134 39
134 41
134 43
134 44
134 46
134 48
134 50
134 52
134 54
134 56
134 58
135 0
135 2
135 4
13

168 44
168 46
168 48
168 50
168 52
168 54
168 56
168 58
169 0
169 2
169 3
169 5
169 7
169 9
169 11
169 13
169 15
169 17
169 19
169 21
169 23
169 25
169 27
169 29
169 31
169 32
169 34
169 36
169 38
169 40
169 42
169 44
169 46
169 48
169 50
169 52
169 54
169 55
169 57
169 59
170 1
170 3
170 5
170 7
170 9
170 11
170 13
170 15
170 16
170 18
170 20
170 22
170 24
170 26
170 28
170 30
170 32
170 34
170 36
170 38
170 39
170 41
170 43
170 45
170 47
170 49
170 51
170 53
170 55
170 57
170 59
171 1
171 2
171 4
171 6
171 8
171 10
171 12
171 14
171 16
171 18
171 20
171 22
171 24
171 26
171 28
171 29
171 31
171 33
171 35
171 37
171 39
171 41
171 43
171 45
171 47
171 49
171 51
171 53
171 55
171 57
171 58
172 0
172 2
172 4
172 6
172 8
172 10
172 12
172 14
172 16
172 18
172 20
172 22
172 23
172 25
172 27
172 29
172 31
172 33
172 35
172 37
172 39
172 41
172 43
172 45
172 47
172 48
172 50
172 52
172 54
172 56
172 58
173 0
173 2
173 4
173 6
173 8
173 10
173 11
173 13
173 15
173 17
173 19
173 21
173 23
173 

206 51
206 53
206 55
206 57
206 59
207 0
207 2
207 4
207 6
207 8
207 10
207 12
207 14
207 16
207 18
207 19
207 21
207 23
207 25
207 27
207 29
207 31
207 33
207 35
207 37
207 39
207 41
207 43
207 45
207 46
207 48
207 50
207 52
207 54
207 56
207 58
208 0
208 2
208 4
208 5
208 7
208 9
208 11
208 13
208 15
208 17
208 19
208 21
208 23
208 24
208 26
208 28
208 30
208 32
208 34
208 36
208 38
208 40
208 42
208 43
208 45
208 47
208 49
208 51
208 53
208 55
208 57
208 59
209 1
209 3
209 4
209 6
209 8
209 10
209 12
209 14
209 16
209 18
209 20
209 22
209 23
209 25
209 27
209 29
209 31
209 33
209 35
209 37
209 39
209 40
209 42
209 44
209 46
209 48
209 50
209 52
209 54
209 56
209 57
209 59
210 1
210 3
210 5
210 7
210 9
210 11
210 13
210 14
210 16
210 18
210 20
210 22
210 24
210 26
210 28
210 30
210 32
210 34
210 35
210 37
210 39
210 41
210 43
210 45
210 47
210 49
210 51
210 53
210 54
210 56
210 58
211 0
211 2
211 4
211 6
211 8
211 9
211 11
211 13
211 15
211 17
211 19
211 21
211 23
211 25
211 27
211 2

244 53
244 55
244 57
244 58
245 0
245 2
245 4
245 6
245 8
245 10
245 12
245 14
245 16
245 18
245 19
245 21
245 23
245 25
245 27
245 29
245 31
245 33
245 35
245 37
245 39
245 41
245 42
245 44
245 46
245 48
245 50
245 52
245 54
245 56
245 58
246 0
246 2
246 4
246 5
246 7
246 9
246 11
246 13
246 15
246 17
246 19
246 21
246 23
246 25
246 27
246 29
246 30
246 32
246 34
246 36
246 38
246 40
246 42
246 44
246 46
246 48
246 50
246 51
246 53
246 55
246 57
246 59
247 1
247 3
247 5
247 7
247 9
247 11
247 13
247 14
247 16
247 18
247 20
247 22
247 24
247 26
247 28
247 30
247 32
247 34
247 35
247 37
247 39
247 41
247 43
247 45
247 47
247 49
247 51
247 53
247 55
247 57
247 59
248 0
248 2
248 4
248 6
248 8
248 10
248 12
248 14
248 16
248 17
248 19
248 21
248 23
248 25
248 27
248 29
248 31
248 33
248 35
248 37
248 38
248 40
248 42
248 44
248 46
248 48
248 50
248 52
248 54
248 55
248 57
248 59
249 1
249 3
249 5
249 7
249 9
249 11
249 13
249 15
249 16
249 18
249 20
249 22
249 24
249 26
249 28
249 30
249 

283 2
283 4
283 6
283 8
283 10
283 12
283 14
283 16
283 17
283 19
283 21
283 23
283 25
283 27
283 29
283 31
283 33
283 34
283 36
283 38
283 40
283 42
283 44
283 46
283 48
283 50
283 52
283 53
283 55
283 57
283 59
284 1
284 3
284 5
284 7
284 8
284 10
284 12
284 14
284 16
284 18
284 20
284 22
284 24
284 25
284 27
284 29
284 31
284 33
284 35
284 37
284 39
284 41
284 43
284 45
284 46
284 48
284 50
284 52
284 54
284 56
284 58
285 0
285 2
285 4
285 6
285 8
285 9
285 11
285 13
285 15
285 17
285 19
285 21
285 23
285 25
285 26
285 28
285 30
285 32
285 34
285 36
285 38
285 40
285 42
285 44
285 46
285 47
285 49
285 51
285 53
285 55
285 57
285 59
286 1
286 3
286 5
286 6
286 8
286 10
286 12
286 14
286 16
286 18
286 20
286 22
286 24
286 26
286 27
286 29
286 31
286 33
286 35
286 37
286 39
286 41
286 43
286 45
286 47
286 49
286 51
286 52
286 54
286 56
286 58
287 0
287 2
287 4
287 6
287 8
287 10
287 12
287 14
287 16
287 17
287 19
287 21
287 23
287 25
287 27
287 29
287 31
287 33
287 34
287 36
287 38
287